In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split


from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_curve, auc

import pandas as pd
import numpy as np

In [2]:
# data = pd.read_csv("transformed_dataset.csv")
# data['NoShow'].value_counts()

In [4]:

class ToTensor:

    def __call__(self, data):
        if not isinstance(data, tuple):
            return torch.from_numpy(data)
        X, y = data
        return torch.from_numpy(X), torch.from_numpy(y)


class MyData(Dataset):

    def __init__(self, transforms=None):
        data = pd.read_csv("transformed_dataset.csv")
        self.X = data.drop(columns=['NoShow']).to_numpy(dtype=np.float32)
        self.y = data['NoShow'].to_numpy(dtype=np.float32).reshape(-1, 1)
        self.transforms = transforms

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, index):
        sample = self.X[index], self.y[index]

        if self.transforms:
            sample = self.transforms(sample)

        return sample

class FocalLoss(nn.Module):

    def __init__(self, alpha=0.5, gamma=2):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        #input is logits, before applying sigmoid
        bce_loss = nn.functional.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        p_t = torch.exp(-bce_loss)

        focal_loss = (1-p_t)**self.gamma * bce_loss

        weights = self.alpha*targets  + (1-self.alpha)*(1-targets)
        weights = weights.to(focal_loss.device)

        return (focal_loss*weights).mean()
        

        

In [101]:
data = MyData(transforms=ToTensor())

input_size = data[0][0].shape[0] #first training example, get features, then get number of features
output_size= 1

train, test = random_split(data, [0.7, 0.3])
train_loader = DataLoader(train, batch_size=1024, shuffle=True)
test_loader = DataLoader(test, batch_size=len(test), shuffle=False)
# data[0][0].shape


In [102]:
model = nn.Sequential(
    nn.Linear(input_size, 64),
    # nn.Dropout(p=0.4),
    nn.ReLU(),
    nn.Linear(64, 32),
    # nn.Dropout(p=0.3),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 16),
    nn.ReLU(),
    nn.Linear(16, 8),
    nn.ReLU(),
    nn.Linear(8, 8),
    nn.ReLU(),
    nn.Linear(8, 1),
)

# loss = nn.BCEWithLogitsLoss(weight=torch.Tensor([3.0]))
loss = FocalLoss(alpha=0.6, gamma=2.0)
optimizer = optim.Adam(model.parameters(), lr=1e-2, weight_decay=1e-2)

In [103]:
# model = model.to('cuda')
# loss = loss.to('cuda')
# # optimizer = optimizer.to('cuda')

In [105]:
for epoch in range(3):
    total_training_loss = 0
    
    for iteration, (features, labels) in enumerate(train_loader):

        output = model(features)
        loss_value = loss(output, labels)

        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()

        total_training_loss += loss_value.item()

        if (iteration%50==0):
            x_test, y_test = next(iter(test_loader))
            with torch.no_grad():
                test_output = torch.sigmoid(model(x_test))
                
            precision, recall, thresholds = precision_recall_curve(y_test, test_output)
            # print(f"")
            maxf1 = 2*np.max(precision*recall/(precision+recall+1e-9))
            
            
            print(f"Epoch: {epoch+1}, iteration: {iteration+1}, loss: {loss_value.item():.4f}, Test PR-AUC: {auc(recall, precision):.4f}, max-f1: {maxf1:.4f}")
            

    print(f"overall epoch {epoch+1} loss: {total_training_loss/iteration}")
    
        

Epoch: 1, iteration: 1, loss: 0.0650, Test PR-AUC: 0.5996, max-f1: 0.3323
Epoch: 1, iteration: 51, loss: 0.0672, Test PR-AUC: 0.5996, max-f1: 0.3323
overall epoch 1 loss: 0.0670233459273974
Epoch: 2, iteration: 1, loss: 0.0657, Test PR-AUC: 0.5996, max-f1: 0.3323
Epoch: 2, iteration: 51, loss: 0.0634, Test PR-AUC: 0.5996, max-f1: 0.3323
overall epoch 2 loss: 0.06700796609123548
Epoch: 3, iteration: 1, loss: 0.0663, Test PR-AUC: 0.5996, max-f1: 0.3323
Epoch: 3, iteration: 51, loss: 0.0682, Test PR-AUC: 0.5996, max-f1: 0.3323
overall epoch 3 loss: 0.06700376590092977


In [92]:
with torch.no_grad():
    test_output = torch.sigmoid(model(x_test))
    
    precision, recall, thresholds = precision_recall_curve(y_test.cpu(), test_output.cpu())
    print("max-", f1_score(y_test.cpu(), (test_output>0.5).cpu(), average='weighted'))
            
    # print(f"")
    maxf1 = 2*np.max(precision*recall/(precision+recall+1e-9))
    print(maxf1)
    print(auc(recall, precision))

max- 0.7034276760011564
0.4321233976729597
0.3071265445367033


In [32]:
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_curve

# f1_score(y_test, y_pred>=0.365, average='weighted')

In [10]:
import psutil
psutil.virtual_memory().used

14527045632

In [11]:
torch.cuda.memory_allocated()

0

In [12]:
### THIS PART OF CODE WAS GENERATED USING GEMINI

import psutil
import os

# Get the current process
process = psutil.Process(os.getpid())

# Get memory info for this specific process
# .rss (Resident Set Size) is often a good measure for actual physical RAM used by the process.
# .vms (Virtual Memory Size) is the total virtual address space, can be much larger.
mem_info = process.memory_info()
rss_bytes = mem_info.rss
vms_bytes = mem_info.vms

print(f"Current script - RSS: {rss_bytes / (1024 * 1024):.2f} MB") # Resident Set Size
print(f"Current script - VMS: {vms_bytes / (1024 * 1024):.2f} MB") # Virtual Memory Size

# For total system usage (what you were using before)
total_system_used_bytes = psutil.virtual_memory().used
print(f"Total system RAM used: {total_system_used_bytes / (1024 * 1024 * 1024):.2f} GB")

Current script - RSS: 629.91 MB
Current script - VMS: 1796.74 MB
Total system RAM used: 13.53 GB
